In [ ]:
import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from tqdm import tqdm
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, LSTM

Using TensorFlow backend.
C:\Users\user\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\user\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\user\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\user\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:529

In [2]:
data_path = './'
train = pd.read_csv(data_path+'train.csv')
feat = np.load('feat.npy', allow_pickle= True)
path = np.load('path.npy')
test = pd.read_csv(data_path +'test.csv')

In [37]:
df_np = pd.DataFrame()
df_np['feat'] = feat
df_np['path'] = path
df_np
df_train = pd.merge(df_np, train, on = 'path')
df_test = pd.merge(df_np, test, on = 'path')

preprocessing real testset and feature extraction

In [27]:
def funct_test():

    lmfcc = []
    lmfcc_mean = []
    lmfcc_std = []
    lstft = []
    lchroma = []
    path_names = []
    lmel = []
    lcontrast = []
    ltonnetz = []
    
    
    for wav in os.listdir(data_path + '/class_samples/'):
              
        if wav in test['path'].values:
            sound, sr = librosa.load(data_path + '/class_samples/' + wav, mono=True, sr=None)
            sound_str = librosa.effects.time_stretch(sound, 0.5)
            stft = np.abs(librosa.stft(sound_str))
            chroma = librosa.feature.chroma_stft(S=stft, sr=sr)
            mel = np.mean(librosa.feature.melspectrogram(sound_str, sr=16000).T,axis=0)
            contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=16000).T,axis=0)
            tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(sound_str), sr=16000).T,axis=0)
            mfcc = librosa.feature.mfcc(sound_str, sr=16000, n_mfcc=13)
            mfcc_mean = np.mean(mfcc)
            mfcc_std = np.std(mfcc)
            
            path_names.append(wav)
            lstft.append(np.mean(stft))
            lchroma.append(np.mean(chroma))
            lmfcc.append(mfcc)
            lmfcc_mean.append(mfcc_mean)
            lmfcc_std.append(mfcc_std)
            lcontrast.append(np.mean(contrast))
            lmel.append(np.mean(mel))
            ltonnetz.append(np.mean(tonnetz))
            
         
          
    return (path_names, lmfcc, lmfcc_mean, lmfcc_std, lstft, lchroma, lcontrast, lmel, ltonnetz)

In [28]:
b = funct_test()
print(len(b))
print(b[8])

9
[0.001427957970076773, 0.009446872888140539, 0.001214988255603058, 0.017388306797007216, 0.0011864425270624987, 0.007165589102509226, -0.007028336054776519, 0.0036018177505590123, -0.012124548700990181, 0.010857408660042452, -0.010888695792875991, 0.00611783593137688, -0.014318993200171626, -0.016278647668317946, -0.004488496092601921, -0.0066781726981452435, 0.020522662458118773, -0.00693800222875568, 0.012591859202314206, -0.015721342868708858]


In [29]:
df_test = pd.DataFrame()
df_test['path']=b[0]
df_test['mfccs']=b[1]
df_test['std']=b[3]
df_test['mean']=b[2]
df_test['stft']=b[4]
df_test['chroma']=b[5]
df_test['contrast']=b[6]
df_test['mel']=b[7]
df_test['tonnetz']=b[8]
print(len(df_test))

20


In [30]:
df_test.to_csv('./real_testset.csv', index = None, header=True)

In [31]:
real_testset = pd.read_csv('real_testset.csv')

In [32]:
X_realtest = real_testset.iloc[:,[2,3,4,5,6,7,8]].values

In [33]:
print(X_realtest)

[[ 9.84537469e+01 -2.65369148e+01  3.39531809e-01  5.59725271e-01
   2.32662099e+01  4.09553240e-01  1.42795797e-03]
 [ 1.14542354e+02 -3.11992400e+01  2.31030926e-01  4.31857581e-01
   2.13907789e+01  1.63502928e+00  9.44687289e-03]
 [ 1.13671649e+02 -2.22211450e+01  1.30528942e-01  6.87913217e-01
   1.85424354e+01  7.72524881e-02  1.21498826e-03]
 [ 1.04006474e+02 -1.86801195e+01  1.74299106e-01  7.19819399e-01
   1.97138816e+01  1.90450957e-01  1.73883068e-02]
 [ 1.19335912e+02 -2.47439056e+01  4.29659903e-01  5.96932950e-01
   2.18139294e+01  9.75148900e-01  1.18644253e-03]
 [ 1.19436174e+02 -2.51633987e+01  2.94347376e-01  6.26856018e-01
   1.98660701e+01  7.93304275e-01  7.16558910e-03]
 [ 1.06464199e+02 -2.72209522e+01  4.05274510e-01  5.18832131e-01
   1.96147207e+01  1.28028228e+00 -7.02833605e-03]
 [ 1.26759537e+02 -3.10089230e+01  2.08659172e-01  6.22409695e-01
   1.97153888e+01  7.73492690e-01  3.60181775e-03]
 [ 1.19135785e+02 -2.76837842e+01  2.11839646e-01  6.24612953e-0

preprocessing train dataset and feature extraction

In [34]:
def funct_np():
   
    lmfcc = []
    lmfcc_mean = []
    lmfcc_std = []
    lstft = []
    lchroma = []
    path_names = []
    path_words = []
    lmel = []
    lcontrast = []
    ltonnetz = []
    
    for wav in os.listdir(data_path + '/class_samples/'):
  
        if wav in train['path'].values:
            sound, sr = librosa.load(data_path + '/class_samples/' + wav, mono=True, sr=None)
            sound_str = librosa.effects.time_stretch(sound, 0.5)
            stft = np.abs(librosa.stft(sound_str))
            chroma = librosa.feature.chroma_stft(S=stft, sr=sr)
            mel = np.mean(librosa.feature.melspectrogram(sound_str, sr=16000).T,axis=0)
            contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=16000).T,axis=0)
            tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(sound_str), sr=16000).T,axis=0)
            mfcc = librosa.feature.mfcc(sound_str, sr=16000, n_mfcc=13)
            mfcc_mean = np.mean(mfcc)
            mfcc_std = np.std(mfcc)
                       
            get_val = train['path'].isin([wav])
            idx = list(get_val[get_val==True].index) 
            label = train['word'][idx[0]]
           
            path_words.append(label)  
            path_names.append(wav)
            lstft.append(np.mean(stft))
            lchroma.append(np.mean(chroma))
            lmfcc.append(mfcc)
            lmfcc_mean.append(mfcc_mean)
            lmfcc_std.append(mfcc_std)
            lcontrast.append(np.mean(contrast))
            lmel.append(np.mean(mel))
            ltonnetz.append(np.mean(tonnetz))

       
    return (path_names, lmfcc, lmfcc_mean, lmfcc_std, lstft, lchroma, lmel, lcontrast, ltonnetz, path_words)

In [35]:
a = funct_np()

C:\Users\Cigdem\Anaconda3\lib\site-packages\librosa\core\pitch.py:145: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn('Trying to estimate tuning from empty frequency set.')


In [54]:
print(len(a[9]))

224


In [56]:
dataframe = pd.DataFrame()
dataframe['path']=a[0]
dataframe['mfccs']=a[1]
dataframe['std']=a[3]
dataframe['mean']=a[2]
dataframe['stft']=a[4]
dataframe['chroma']=a[5]
dataframe['contrast']=a[6]
dataframe['mel']=a[7]
dataframe['tonnetz']=a[8]
dataframe['word']=a[9]

In [57]:
dataframe.to_csv('./sumdataframe.csv', index = None, header=True)

In [58]:
dataset = pd.read_csv('sumdataframe.csv')

In [59]:
X = dataset.iloc[:,[2,3,4,5,6,7,8]].values
y = dataset.iloc[:, 9].values

In [60]:
print(len(np.unique(y)))
print(len(y))

35
224


In [61]:
# encode class values as integers
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
encoder = LabelEncoder()
encoder.fit(y)
encoded_Y = encoder.transform(y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)
#sample_pred = dummy_y.argmax(1) #reversing to an integer

In [62]:
#Splitting
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, dummy_y, test_size = 0.2, random_state = 7)

In [63]:
#Feature scaling
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train) #we fit and transform for the train set, the test set is only transformed
X_test = sc_X.transform(X_test)

In [64]:
print(y_test)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [65]:
print(X_train.shape, X_train.size)
print(y_test.shape)
print(X_test.shape, X_test.size)
print(y_train.shape)

(179, 7) 1253
(45, 35)
(45, 7) 315
(179, 35)


In [66]:
#Initialize ann
classifier = Sequential()

In [67]:
#Input layer and hidden layer -- here tuning was applied
classifier.add(Dense(output_dim=21, init='uniform', activation='relu', input_dim=7))

C:\Users\Cigdem\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", input_dim=7, units=21, kernel_initializer="uniform")`
  


In [68]:
classifier.add(Dense(output_dim=21, init='uniform', activation='relu'))

C:\Users\Cigdem\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=21, kernel_initializer="uniform")`
  """Entry point for launching an IPython kernel.


In [69]:
classifier.add(Dense(output_dim=35, init='uniform', activation='softmax'))

C:\Users\Cigdem\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=35, kernel_initializer="uniform")`
  """Entry point for launching an IPython kernel.


In [70]:
classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [71]:
classifier.fit(X_train, y_train, batch_size = 10, nb_epoch =100)

C:\Users\Cigdem\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.


Epoch 1/100
179/179 [==============================] - ETA: 6s - loss: 3.5555 - acc: 0.0000e+0 - 0s 3ms/step - loss: 3.5545 - acc: 0.0391
Epoch 2/100
179/179 [==============================] - ETA: 0s - loss: 3.5521 - acc: 0.100 - 0s 295us/step - loss: 3.5484 - acc: 0.0615
Epoch 3/100
179/179 [==============================] - ETA: 0s - loss: 3.5403 - acc: 0.100 - 0s 273us/step - loss: 3.5395 - acc: 0.0615
Epoch 4/100
179/179 [==============================] - ETA: 0s - loss: 3.5537 - acc: 0.0000e+0 - 0s 301us/step - loss: 3.5219 - acc: 0.0615
Epoch 5/100
179/179 [==============================] - ETA: 0s - loss: 3.4194 - acc: 0.100 - ETA: 0s - loss: 3.4941 - acc: 0.058 - 0s 440us/step - loss: 3.4897 - acc: 0.0615
Epoch 6/100
179/179 [==============================] - ETA: 0s - loss: 3.4984 - acc: 0.0000e+0 - 0s 273us/step - loss: 3.4468 - acc: 0.0615
Epoch 7/100
179/179 [==============================] - ETA: 0s - loss: 3.4013 - acc: 0.0000e+0 - 0s 290us/step - loss: 3.3961 - acc: 0.0

In [72]:
y_pred = classifier.predict(X_test)
predictions = y_pred.argmax(1)

print(encoder.inverse_transform(predictions))


C:\Users\Cigdem\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


['three' 'eight' 'three' 'marvin' 'three' 'three' 'happy' 'two' 'three'
 'stop' 'four' 'dog' 'four' 'nine' 'three' 'down' 'marvin' 'nine' 'off'
 'three' 'one' 'one' 'eight' 'happy' 'three' 'dog' 'three' 'three' 'three'
 'off' 'off' 'three' 'dog' 'three' 'three' 'three' 'four' 'marvin' 'down'
 'eight' 'four' 'down' 'no' 'go' 'on']


In [73]:
y_predreal = classifier.predict(X_realtest)
predictions_real = y_predreal.argmax(1)
words = encoder.inverse_transform(predictions_real)
print(words)


C:\Users\Cigdem\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


['eight' 'eight' 'eight' 'eight' 'eight' 'eight' 'eight' 'eight' 'eight'
 'eight' 'eight' 'eight' 'eight' 'eight' 'eight' 'eight' 'eight' 'eight'
 'eight' 'eight']


In [74]:
result = real_testset
result = result.drop(['mfccs', 'std', 'mean', 'stft', 'chroma', 'contrast', 'mel', 'tonnetz', ], axis=1)

In [75]:
result['word'] = words
result.to_csv("result.csv", sep=",")